In [1]:
import numpy as np
import tweepy
import config
import pyspark
import emoji
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_unixtime, date_format, col, avg, sum, to_timestamp
from pyspark.sql.functions import concat, lit
from pyspark import SparkConf, SparkContext
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import udf
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.types import StringType
import findspark
findspark.init()
import cryptocompare
import yfinance as yf
import pandas as pd
from datetime import datetime, date, time, timedelta
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-darkgrid')
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pickle
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer(stop_words='english')
import csv
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler


# Create a SparkSession
spark = SparkSession.builder.appName("MyApp").getOrCreate()

### part 1: get cryptocurrency

In [28]:
# Fetch BTC trade raw data
today = date.today()
BTC_raw = yf.download('ADA-USD', start=today-timedelta(days=7), end=today) # BTC, ETH, XRP, ADA, DOGE
# BTC_raw.columns = ['date', 'open_price', 'high', 'low', 'close_price', 'adj close', 'volume']

# reset index and move Date column to the first column
BTC_raw = BTC_raw.reset_index().rename(columns={'Date': 'date'})
BTC_raw = BTC_raw.loc[:, ['date', 'Close']]
BTC_raw.head()

l = len(BTC_raw["date"])
a = []
for n in range(l):
    data = [str(BTC_raw["date"][n])[0:10], float(BTC_raw["Close"][n])]
    a.append(data)
    
print(a)

[*********************100%***********************]  1 of 1 completed
[['2023-04-22', 0.3961530029773712], ['2023-04-23', 0.3889999985694885], ['2023-04-24', 0.3839159905910492], ['2023-04-25', 0.3949190080165863], ['2023-04-26', 0.4014979898929596], ['2023-04-27', 0.40984299778938293], ['2023-04-28', 0.4049319922924042], ['2023-04-29', 0.40278398990631104], ['2023-04-30', 0.39735400676727295]]


In [21]:
day_price_data = spark.createDataFrame(a,["date", "close price"])

day_price_data.show()

+----------+-------------------+
|      date|        close price|
+----------+-------------------+
|2023-04-23| 0.3889999985694885|
|2023-04-24| 0.3839159905910492|
|2023-04-25| 0.3949190080165863|
|2023-04-26| 0.4014979898929596|
|2023-04-27|0.40984299778938293|
|2023-04-28| 0.4049319922924042|
|2023-04-29|0.40278398990631104|
+----------+-------------------+



### part 2: get twitter

In [22]:
# Set up Twitter API credentials
consumer_key = config.API_KEY
consumer_secret= config.API_SECRET
access_token= config.ACCESS_TOKEN
access_token_secret = config.ACCESS_TOKEN_SECRET

# Authenticate with Twitter API
auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth)


# Query Twitter API for tweets
data = tweepy.Cursor(api.search_tweets, q="Dogecoin OR DOGE", until="2023-04-30 00:00:00", lang="en", count=100).items(8000)

# Create an empty list to store the processed data
processed_data_list = []

print("fetching twitter data...")

while True:
    try:
        tweet = data.next()
            
        user_date = tweet.created_at
        
        #user_text = emoji.replace_emoji(tweet.text, replace="")
        user_text = tweet.text
        final_data = [user_date, user_text]   
            
        processed_data_list.append(final_data)
      
    except StopIteration:
        break

print("total twitter data:", len(processed_data_list))


fetching twitter data...
total twitter data: 7495


In [23]:
# sentiment analysis model 

analyzer=SentimentIntensityAnalyzer()
def get_vader_score(sentence): 
    compound=analyzer.polarity_scores(sentence)['compound']
    if compound > 0.05: 
        return ['positive',compound]
    elif (compound >= -0.05) and (compound <=0.05): 
        return ['neutral',compound]
    else: 
        return ['negative',compound]

def vader_sentiment(X):
  vader_result=get_vader_score(X)[0]#X.apply(lambda x: get_vader_score(x)[0])
  vader_score = get_vader_score(X)[1]#X.apply(lambda x: get_vader_score(x)[1])
  #vader_score_total = np.mean(vader_score)
  return vader_score


In [24]:
# remove emoji
def remove_emoji(col):
  result = emoji.replace_emoji(col, replace="")
  return result


# Create a DataFrame from the processed data list
twitter_data = spark.createDataFrame(processed_data_list, ["date", "text"])

# start PySpark transform #####################################################################

clean_udf = F.UserDefinedFunction(remove_emoji, T.StringType())
tweets_df_cleaned = twitter_data.withColumn("text", clean_udf("text"))

# remove mention 
cleaned_twitter_data = twitter_data.withColumn("text", regexp_replace('text', "@\s*[A-Za-z0-9_]+", ''))
cleaned_twitter_data = cleaned_twitter_data.withColumn("text", regexp_replace("text", "#\s*[A-Za-z0-9_]+", ""))
# remove retweet
cleaned_twitter_data = cleaned_twitter_data.withColumn("text", regexp_replace('text', "RT : ", ''))

# remove links
cleaned_twitter_data = cleaned_twitter_data.withColumn('text', regexp_replace('text', r"http\S+", ''))
cleaned_twitter_data = cleaned_twitter_data.withColumn('text', regexp_replace('text', r"www.\S+", ''))

# remove next line
cleaned_twitter_data = cleaned_twitter_data.withColumn("text", regexp_replace("text", r"\n", ""))

cleaned_twitter_data = cleaned_twitter_data.withColumn('text', regexp_replace('text', '\s+', ' '))

cleaned_twitter_data = cleaned_twitter_data.select(col('date'), col('text'))

# put sentiment analysis here

# model 1

vader_sentiment_udf = F.UserDefinedFunction(vader_sentiment, T.FloatType())
vader_sentiment_twitter_data = cleaned_twitter_data.withColumn("text", vader_sentiment_udf("text"))

# calculate average

vader_sentiment_twitter_data = vader_sentiment_twitter_data.groupBy(date_format(col("date"), "yyyy-MM-dd").alias("date"))
vader_sentiment_twitter_data = vader_sentiment_twitter_data.agg(avg("text").alias("avg_value"))

In [ ]:

final = vader_sentiment_twitter_data.collect()

In [ ]:
vader_sentiment_twitter_data.show()

In [ ]:
# save csv

with open("C:\\Users\\ee527\\Desktop\\DOGE.csv", 'w', newline='') as f:
    # create the csv writer
    writer = csv.writer(f)
    
    for row in final:
        
        # write a row to the csv file
        writer.writerow(row)

### part 3: integrate two data

In [ ]:
vader_sentiment_twitter_data.show()

In [29]:
merged_df = vader_sentiment_twitter_data.join(day_price_data,"date","left")

merged_df = merged_df.dropna()

merged_df.show()

+----------+--------------------+-------------------+
|      date|           avg_value|        close price|
+----------+--------------------+-------------------+
|2023-04-29|  0.1506818802216289|0.40278398990631104|
|2023-04-28|0.038404610113824726| 0.4049319922924042|
|2023-04-27|-0.00744525705840...|0.40984299778938293|
|2023-04-26| 0.14644978303556977| 0.4014979898929596|
|2023-04-25|  0.2427375049376378| 0.3949190080165863|
|2023-04-24|  0.3055099020250612| 0.3839159905910492|
|2023-04-23| 0.04314303423794154| 0.3889999985694885|
+----------+--------------------+-------------------+

